In [ ]:
import gymnasium as gym 
import torch 
import torch.nn as nn
env  = gym.make("CartPole-v1", render_mode="human")
obs , info = env.reset(seed=42)
obs = torch.tensor(obs) 

class Policy(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.core = nn.Sequential(
            nn.Linear(4,128),
            nn.ReLU(),
            nn.Linear(128,128),
            nn.ReLU(),
            nn.Linear(128,2)
            
        )
    def forward(self, x):
        return self.core(x)
    
policy = Policy()

episodes = 1000
gamma = 0.99


for episode in range(episodes):
    obs , info = env.reset(seed=42)
    obs = torch.tensor(obs)
    done = False 
    observations = []
    actions = []
    rewards = []
    
    while not done :
        logits = policy(obs)
        probs = torch.softmax(logits , dim=-1)
        action = torch.argmax(probs).item()
        obs , reward , done , truncated , info = env.step (action)
        obs = torch.tensor(obs)
        observations.append(obs)
        actions.append(action)
        rewards.append(reward)
        if done or truncated: 
            break 
    Gs = []
    G = 0
    for i in range(len(rewards) - 1 , -1 , -1):
        G = rewards[i] + gamma * G
        Gs.append(G)
    Gs.reverse()
    Gs = torch.tensor(Gs)
    Gs = (Gs - Gs.mean()) / (Gs.std() + 1e-8)

    loss = 0 
    for obs, action , G in zip(observations , actions , Gs):
        logits = policy(obs)
        




        

SyntaxError: incomplete input (858066065.py, line 51)

: 